# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install widgetsnbextension

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = "../output_data/cities.csv"
city_DF = pd.read_csv(csvpath)
city_DF.head()

,City,Country,Temperature (C),Max Temperature (C),Humidity,Cloudiness,Windspeed (mph),Latitude,Longitude
0,Punta Arenas,CL,2.96,3.06,65,20,8.23,-53.1500,-70.9167
1,Cherskiy,RU,5.63,5.63,68,4,4.34,68.7500,161.3000
2,Thompson,CA,18.09,18.09,72,75,1.03,55.7435,-97.8558
3,Butaritari,KI,27.53,27.53,75,12,7.99,3.0707,172.7902
4,Laas,IT,22.23,23.49,89,55,2.28,46.6166,10.7002


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_DF[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
heat_index = city_DF['Humidity'].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# add a marker latyer on heatmap
marker_layer = gmaps.marker_layer(locations)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=heat_index,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# The perfect weather conditions! (Drop all rows that doesnt satisfy conditions):
# Humidity < 60, Temp between 25 to 40 Celcius, Cloudiness < 50

city_DF.drop(city_DF[city_DF["Humidity"] > 60].index, inplace= True)
city_DF.drop(city_DF[(city_DF["Max Temperature (C)"] < 25) | (city_DF["Max Temperature (C)"] > 40)].index, inplace= True)
city_DF.drop(city_DF[city_DF["Cloudiness"] > 50].index, inplace= True)
city_DF.dropna(inplace = True) 

cleaned_city_DF = city_DF
#cleaned_city_DF.head()
#len(cleaned_city_DF['Cloudiness'])

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
from pprint import pprint
hotel_df = cleaned_city_DF

In [7]:
# Set parameters
#target_coordinates = ""
target_radius = 5000 #(Meters)
target_type = "hotel"

# Dictionary of parameters
parameters = {  
    'radius' : target_radius, 
    'type' : target_type, 
    'key' : g_key
}



In [8]:
# Request url to test if works
# hotel_response = requests.get(base_url, params = parameters)
# hotelJSON = hotel_response.json()
# pprint(hotelJSON)

# hotel_response['results'][1]['name']

# Create for loop to search for hotels in every city in DF
for i, row in hotel_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    
    # Change target coordinates each iteration while leaving other params in place
    parameters['location'] = f'{lat}, {long}'
    
    # Use base_url to search for hotels using coords
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and jsonify
    print(f"Results for Index {i}: {row['City']}.")
    hotelJSON = requests.get(base_url, params= parameters).json()
    
    # Extract results
    results = hotelJSON['results'][1]['name']
    
    # Skip if no hotels are found
    try:
        print(f"Nearest hotel within {target_radius} meters is {results}")
        hotel_df.loc[i, "Hotel Name"] = results
    except (KeyError, IndexError):
        print("There was no result.")
              
    print("--------------------------------")
              


Results for Index 21: Guerrero Negro.
Nearest hotel within 5000 meters is COMERBAJA
--------------------------------
Results for Index 26: Alice.
Nearest hotel within 5000 meters is Holiday Inn Express & Suites Alice, an IHG Hotel
--------------------------------
Results for Index 47: Akdepe.
Nearest hotel within 5000 meters is Elwan
--------------------------------
Results for Index 48: Jamestown.
Nearest hotel within 5000 meters is Hampton Inn & Suites Jamestown
--------------------------------
Results for Index 100: Ponta do Sol.
Nearest hotel within 5000 meters is Hotel do Campo
--------------------------------
Results for Index 116: Muros.
Nearest hotel within 5000 meters is BBVA
--------------------------------
Results for Index 117: Tiznit Province.
Nearest hotel within 5000 meters is Terrain de proximité Imjgaguene
--------------------------------
Results for Index 122: Portland.
Nearest hotel within 5000 meters is Embassy Suites by Hilton Portland Downtown
--------------------

Nearest hotel within 5000 meters is Holiday Inn Express & Suites Rolla - Univ of Missouri S&T, an IHG Hotel
--------------------------------
Results for Index 545: Jackson.
Nearest hotel within 5000 meters is Rug Busters Carpet Cleaning & Restoration
--------------------------------
Results for Index 558: Três Lagoas.
Nearest hotel within 5000 meters is Druds Express Hotel
--------------------------------
Results for Index 560: Hailey.
Nearest hotel within 5000 meters is Wood River Inn & Suites
--------------------------------


In [9]:
hotel_df.head(1)

,City,Country,Temperature (C),Max Temperature (C),Humidity,Cloudiness,Windspeed (mph),Latitude,Longitude,Hotel Name
21,Guerrero Negro,MX,27.43,27.43,48,44,3.36,27.9769,-114.0611,COMERBAJA


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))